Install missing modules and imports

In [1]:
import sys
import subprocess
import pkg_resources

required = {'scipy', 'numpy', 'pandas', 'openpyxl'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

import pandas as pd
import numpy as np
import math

Transform to PD

In [2]:
file = 'data.xlsx'
df = [pd.read_excel(file, sheet_name = sheet) for sheet in ['CAC', 'DAX', 'IBEX35TR', 'IBEX', 'SX5E', 'SX5T', 'SMI']]
for i in range(len(df)):
    df[i] = df[i].drop(0)

Helpers

In [3]:
def MSE(A, B):
    err = 0
    for i in range(len(A)):
        payoff = max(A[i] - K, 0)
        err += (payoff - B[i])**2
    return err

def g(x):
    return max(x-K, 0)


Split data train test

In [4]:
train_per = 0.6
train = [df[i].iloc[:math.floor(train_per*df[i].shape[0]) - math.floor(train_per*df[i].shape[0]) % 10] for i in range(len(df))]
test = [df[i].iloc[math.floor(train_per*df[i].shape[0]) - math.floor(train_per*df[i].shape[0]) % 10:(df[i].shape[0]//10)*10] for i in range(len(df))]
K = 5000

Méthode I : Split en dizaine

In [5]:
train_D = [[train[i][j:j+10] for j in range(0,len(train[i]),10)] for i in range(len(train))]
train_CAC40 = train_D[0]
train_CAC40[0]

,DATES,DELTA_DAYS,PX_LAST,PX_HIGH,PX_LOW,PX_VOLUME,S1/S,D_S
1,2000-11-22,1,5944.70,6049.60,5937.37,47470800,0.977583,-136.32
2,2000-11-23,1,6053.04,6055.21,5933.08,37599950,1.018225,108.34
3,2000-11-24,1,6145.65,6162.88,5989.13,47878450,1.015300,92.61
4,2000-11-27,3,6171.33,6254.17,6167.45,40542910,1.004179,25.68
5,2000-11-28,1,6069.22,6172.77,6055.04,68980960,0.983454,-102.11
6,2000-11-29,1,6060.65,6091.58,5978.96,51022320,0.998588,-8.57
7,2000-11-30,1,5928.08,6015.86,5893.54,61617140,0.978126,-132.57
8,2000-12-01,1,5928.50,6023.11,5915.33,49440530,1.000071,0.42
9,2000-12-04,3,5791.51,5933.31,5763.18,37464750,0.976893,-136.99
10,2000-12-05,1,5994.89,5994.89,5836.83,52939960,1.035117,203.38


Calcul de Ku, Kd et lamda

In [24]:
def train(L):
    Ku = np.ones(len(L[0]))
    Kd = np.ones(len(L[0]))

    for i in range(len(L)):
        for j in range(len(L[i])):
            if L[i].iloc[j]['S1/S'] > Ku[j] :
                Ku[j] = L[i].iloc[j]['S1/S']
            if L[i].iloc[j]['S1/S'] < Kd[j] :
                Kd[j] = L[i].iloc[j]['S1/S']

    lamda = [(1 - Kd[i])/(Ku[i] - Kd[i]) for i in range(len(Ku))]

    V = []
    VT = []
    ST = []
    for i in range(len(L)):
        D = []
        for j in range(len(L[i])):
            if j == 0:
                D.append(L[i].iloc[j]['PX_LAST'])
            else:
                Teta = (g(Ku[j]*L[i].iloc[j]['PX_LAST'])*lamda[j] + g(Kd[j]*L[i].iloc[j]['PX_LAST'])*(1-lamda[j]))/((Ku[j]-Kd[j])*L[i].iloc[j]['PX_LAST'])
                D.append(D[-1] + Teta*L[i].iloc[j]['D_S'])
        V.append(D)
        VT.append(D[-1])
        ST.append(L[i].iloc[len(L[i])-1]['PX_LAST'])

    return (MSE(ST, VT), Ku, Kd)


In [42]:
def test(L, Ku, Kd):
    lamda = [(1 - Kd[i])/(Ku[i] - Kd[i]) for i in range(len(Ku))]

    V = []
    VT = []
    ST = []
    for i in range(len(L)):
        D = []
        for j in range(len(L[i])):
            if j == 0:
                D.append(L[i].iloc[j]['PX_LAST'])
            else:
                Teta = (g(Ku[j]*L[i].iloc[j]['PX_LAST'])*lamda[j] + g(Kd[j]*L[i].iloc[j]['PX_LAST'])*(1-lamda[j]))/((Ku[j]-Kd[j])*L[i].iloc[j]['PX_LAST'])
                D.append(D[-1] + Teta*L[i].iloc[j]['D_S'])
        V.append(D)
        VT.append(D[-1])
        ST.append(L[i].iloc[len(L[i])-1]['PX_LAST'])
        print(ST[-1])
        print(VT[-1])

    return MSE(ST, VT)

In [47]:
CAC_train = train_CAC40[:50]
CAC_test = train_CAC40[200:202]

val, Ku, Kd = train(CAC_train)

Val = test(CAC_test, Ku, Kd)

print(Val)

4386.35
4408.74
4332.79
4448.33
39224628.17649999


4386.35
4408.74
4332.79
4448.33
39224628.17649999